# Вебинар 4. Домашнее задание

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import coo_matrix, csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
data = pd.read_csv('02_Data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                     'product_id': 'item_id'},
            inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('02_Data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
# n_items_before = data_train['item_id'].nunique()

# data_train = prefilter_items(data_train, item_features, take_n_popular=5000)

# n_items_after = data_train['item_id'].nunique()
# print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(2)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
bm25_user_item_matrix = bm25_weight(sparse_user_item.T).T  # Применяется к item-user матрице ! (coo-matrix)

In [9]:
%%time

model = AlternatingLeastSquares(factors=20,
                                regularization=0.001,
                                iterations=15,
                                calculate_training_loss=True,
                                num_threads=4,
                                random_state=1234)

model.fit(bm25_user_item_matrix.T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 1min 4s, sys: 49.6 s, total: 1min 54s
Wall time: 10.1 s


-----

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [10]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix #, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight #, tfidf_weight


# class MainRecommender:
#     """Рекоммендации, которые можно получить из ALS
    
#     Input
#     -----
#     data: pd.DataFrame
#         Матрица взаимодействий user-item (факта покупки)
        
#     weighting: bool, default True
#         Флаг для выполнения взвешивания данных (BM25)
#     """
    
#     def __init__(self, data:pd.DataFrame, weighting:bool=True):
        
#         # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать

#         self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
#         self.sparse_user_item = csr_matrix(self.user_item_matrix)
        
#         self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)

#         if weighting:
#             self.bm25_user_item_matrix = bm25_weight(self.sparse_user_item.T).T # csr-matrix

#         self.model = self.fit(self.bm25_user_item_matrix)
        
#         self.own_recommender = self.fit_own_recommender(self.bm25_user_item_matrix)

#     @staticmethod
#     def prepare_matrix(data:pd.DataFrame) -> pd.DataFrame:
#         # your_code
#         user_item_matrix = pd.pivot_table(data, index='user_id', columns='item_id',
#                                           values='quantity', # Можно пробовать другие варианты
#                                           aggfunc='count', 
#                                           fill_value=0
#                                          )

#         user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
        
#         return user_item_matrix
    
#     @staticmethod
#     def prepare_dicts(user_item_matrix:pd.DataFrame) -> (dict, dict, dict, dict):
#         """Подготавливает вспомогательные словари"""
        
#         userids = user_item_matrix.index.values
#         itemids = user_item_matrix.columns.values

#         matrix_userids = np.arange(len(userids))
#         matrix_itemids = np.arange(len(itemids))

#         id_to_itemid = dict(zip(matrix_itemids, itemids))
#         id_to_userid = dict(zip(matrix_userids, userids))

#         itemid_to_id = dict(zip(itemids, matrix_itemids))
#         userid_to_id = dict(zip(userids, matrix_userids))
        
#         return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
#     def get_top_n_users_items(self, user_id, N=5):
#         df = self.user_item_matrix.loc[user_id, :].reset_index().groupby('item_id').sum()
#         if N > 0:
#             df = df.sort_values(by=user_id, ascending=False).head(N).reset_index()
#             return df['item_id'].tolist()
#         elif N == -1:
#             df = df.sort_values(by=user_id, ascending=False).reset_index()
#             df = df[df[user_id] > 0]
#             return df['item_id'].tolist()
    
#     @staticmethod
#     def fit(bm25_user_item_matrix:csr_matrix,
#             n_factors:int=20, regularization:float=0.001,
#             iterations:int=15, num_threads:int=4
#            ) -> 'ALS_Model':
        
#         """Обучает ALS"""
        
#         model = AlternatingLeastSquares(factors=n_factors, 
#                                         regularization=regularization,
#                                         iterations=iterations,  
#                                         num_threads=num_threads,
#                                         calculate_training_loss=True,
#                                         random_state=1234)
        
#         model.fit(bm25_user_item_matrix.T.tocsr(),
#                  show_progress=False)
        
#         return model

#     @staticmethod
#     def fit_own_recommender(bm25_user_item_matrix:csr_matrix) -> 'ItemItem_Model':
#         """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
#         own_recommender = ItemItemRecommender(K=1, num_threads=4)
#         own_recommender.fit(bm25_user_item_matrix.T.tocsr(),
#                  show_progress=False)
        
#         return own_recommender
    
#     def get_similar_items_recommendation(self, user_id:int, N:int=5) -> list:
#         """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
#         # your_code
#         items_id = self.get_top_n_users_items(user_id, N=N)
#         rec = []
#         for item in items_id:
#             rec.append(self.id_to_itemid[self.model.similar_items(self.itemid_to_id[item], N=2)[1][0]])

#         assert len(rec) == N, 'Количество рекомендаций != {}'.format(N)
#         return rec
    
#     def get_similar_users_recommendation(self, user_id:int, N:int=5) -> list:
#         """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
#         # your_code
#         top_n_nearest_users = [self.id_to_userid[uidx] for uidx, _ in self.model.similar_users(self.userid_to_id[user_id], N=5+1)[1:]]
#     #     print(top_n_nearest_users)

#         top_items_nearest_users = []
#         for uidx in top_n_nearest_users:
#             top_items_nearest_users.append(self.get_top_n_users_items(uidx, N=-1))
#     #     print([len(lst) for lst in top_items_nearest_users])

#         rec = []
#         while len(rec) < N:
#             for i in range(len(top_items_nearest_users)):
#                 item = top_items_nearest_users[i].pop(0)
#                 if len(rec) < N and item not in rec:
#                     rec.append(item)

#         assert len(rec) == N, 'Количество рекомендаций != {}'.format(N)
#         return rec

----

Проверка, что все работает

In [11]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [12]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [13]:
my_recommender = MainRecommender(data)

In [14]:
my_recommender.get_similar_items_recommendation(443, N=5)

[8011282, 1038072, 1004906, 9221021, 850004]

In [15]:
item_features[item_features['item_id'].isin([871332, 16730819, 9302594, 2612148, 1373649])]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
11855,871332,69,GROCERY,Private,FRUIT - SHELF STABLE,FRUIT BOWL AND CUPS,4.5 OZ
44467,1373649,1232,GROCERY,National,REFRGRATD JUICES/DRNKS,DAIRY CASE JUICE DRNK UNDER 10,64 OZ
51020,2612148,2,GROCERY,National,BOTTLE DEPOSITS,BOTTLE DEPOSITS,
65691,9302594,1194,GROCERY,National,REFRGRATD JUICES/DRNKS,DAIRY CASE 100% PURE JUICE - O,64 OZ
90297,16730819,6402,DRUG GM,National,BOOKSTORE,HARDBACK/TRADE BEST SELLER,


In [16]:
my_recommender.get_similar_users_recommendation(443, N=5)

[1082185, 1058997, 995242, 1029272, 1070820]

In [17]:
item_features[item_features['item_id'].isin([1082185, 1058997, 995242, 1029272, 1070820])]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
25754,995242,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,
29591,1029272,69,GROCERY,Private,CANNED JUICES,PRUNE JUICE (OVER 50% JUICE),64 OZ
32948,1058997,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,
34259,1070820,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA
35576,1082185,2,PRODUCE,National,TROPICAL FRUIT,BANANAS,40 LB


In [18]:
my_recommender.model.similar_items(11435, N=5)

[(11435, 1.0000001),
 (20991, 0.85587233),
 (460, 0.85587233),
 (82, 0.85587233),
 (3462, 0.85587233)]

In [19]:
model.similar_items(11435, N=5)

[(11435, 1.0000001),
 (38390, 0.8378864),
 (27027, 0.8102505),
 (40260, 0.80259746),
 (14919, 0.7984483)]